In [1]:
# run each cell with shift-enter
# run this first to get dependencies

from watson_developer_cloud import ToneAnalyzerV3
from watson_developer_cloud import NaturalLanguageUnderstandingV1
from watson_developer_cloud import WatsonApiException
import json
from watson_developer_cloud.natural_language_understanding_v1 \
  import Features, EntitiesOptions, KeywordsOptions

import numpy as np

In [2]:
# init the nlu and ta

natural_language_understanding = NaturalLanguageUnderstandingV1(
  username='deb24ba1-c4b9-4e65-bea9-6083fbeca2a0',
  password='R0ghN87kqYnJ',
  version='2018-03-16')

tone_analyzer = ToneAnalyzerV3(
    version='2018-03-16',
    username='504244a6-b618-4f5a-97d9-5ac26e0ff3ba',
    password='dkNpTqbF8YkG',
    url='https://gateway.watsonplatform.net/tone-analyzer/api'
)

In [3]:
# test response from nlu

#response = natural_language_understanding.analyze(
#  text=text,
#  features=Features(
#    entities=EntitiesOptions(
#      emotion=True,
#      sentiment=True,
#      limit=2),
#    keywords=KeywordsOptions(
#      emotion=True,
#      sentiment=True,
#      limit=2))).get_result()

#print(json.dumps(response, indent=2))

In [4]:
# test response from ta

#try:
#  tone_analysis = tone_analyzer.tone(
#      {'text': text},
#      'application/json').get_result()
#  print(json.dumps(tone_analysis, indent=2))
#except WatsonApiException as ex:
#    print ("Method failed with status code " + str(ex.code) + ": " + ex.message)

In [5]:
def getAnal(text):
  try:
    response = natural_language_understanding.analyze(
      text=text,
      features=Features(
        entities=EntitiesOptions(
          emotion=True,
          sentiment=True,
          limit=2),
        keywords=KeywordsOptions(
          emotion=True,
          sentiment=True,
          limit=2))).get_result()
    keywords = response["keywords"]
    numKeywords = len(keywords)
    if numKeywords == 0:
      print("no keywords")
      return np.zeros(6)
    sentiments = np.array([keyword["sentiment"]["score"] * keyword["relevance"] for keyword in keywords])
    totalSentiment = np.sum(sentiments) / numKeywords

    emotionNames = ['sadness', 'joy', 'fear', 'disgust', 'anger']
    emotions = np.array([np.array([keyword["emotion"][name] for name in emotionNames]) for keyword in keywords])
    totalEmotions = np.sum(emotions, 0) / numKeywords

    features = np.insert(totalEmotions, 0, totalSentiment, axis=0)
    return features
  except:
    print("error")
    return np.zeros(6)



In [6]:
def analyzeBlock(textBlocks):
  final = [];
  for i, t in enumerate(textBlocks):
    if i % 100 == 0:
      print("{} steps".format(i))
    final.append(getAnal(t));
  return final;

In [ ]:
import csv

texts=[]
with open("yelp_training_data_short.csv") as csv_file:
  csv_reader = csv.reader(csv_file, delimiter=',')
  line_count = 0
  for row in csv_reader:
    if line_count > 25000:
        texts.append(row[0])
    if line_count > 50000:
        break
    line_count += 1
print(['sentiment', 'sadness', 'joy', 'fear', 'disgust', 'anger'])
analysedData = analyzeBlock(texts)

['sentiment', 'sadness', 'joy', 'fear', 'disgust', 'anger']
0 steps
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
100 steps
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
erro

In [ ]:
analysedDataNp = np.array(analysedData)

In [ ]:
print(analysedDataNp.shape)
## need to append the labels

In [ ]:
labels=[]
with open("yelp_training_data_short.csv") as csv_file:
  csv_reader = csv.reader(csv_file, delimiter=',')
  line_count = 0
  for row in csv_reader:
    labels.append(row[1])
    line_count += 1
    if line_count > 25000:
      break
labelsNp = np.array(labels)[:,np.newaxis]
print(labelsNp.shape)

In [ ]:
dataWithLabels = np.append(analysedDataNp, labelsNp, 1).astype(float)
print(dataWithLabels.shape)

In [ ]:
finalData = dataWithLabels[(dataWithLabels[:, 0] != 0) & (dataWithLabels[:, 1] != 0) & (dataWithLabels[:, 2] != 0) & (dataWithLabels[:, 3] != 0) & (dataWithLabels[:, 4] != 0) & (dataWithLabels[:, 5] != 0)]
print(finalData.shape)

In [ ]:
myFile = open('finalData.csv', 'w')
with myFile:
    writer = csv.writer(myFile)
    writer.writerows(finalData)